# 1. 概述
## Paint 的 API 大致可以分为 4 类
- 颜色
- 效果
- drawText 相关
- 初始化

---

# 2. 颜色
- Canvas 绘制的内容，有三层对颜色的处理

![image](paint1_page1.jpg)

## 2.1 基本颜色
- Paint 设置颜色的方法有两种
  - 一种是直接用 Paint.setColor 或者 Paint.setARGB() 来设置颜色
  - 一种是使用 Shader 来指定着色方案。

![image](paint1_page2.jpg)

### 2.1.1 直接设置颜色
  - setColor(int color)
  - setARGB(int a, int r, int g, int b)

```
paint.setColor(Color.parseColor("#009688"));  
canvas.drawRect(30, 30, 230, 180, paint);

paint.setColor(Color.parseColor("#FF9800"));  
canvas.drawLine(300, 30, 450, 180, paint);

paint.setColor(Color.parseColor("#E91E63"));  
canvas.drawText("HenCoder", 500, 130, paint);
```

```
paint.setARGB(100, 255, 0, 0);  
canvas.drawRect(0, 0, 200, 200, paint);  
paint.setARGB(100, 0, 0, 0);  
canvas.drawLine(0, 0, 200, 200, paint);
```

---

### 2.1.2 setShader(Shader shader)
#### 2.1.2.1 LinearGradient：线性渐变
- 设置两个点和两个颜色
  - x0、y0、x1、y1：渐变的两个端点的位置 
  - color0、color1 是端点的颜色 
  - tile：端点范围之外的着色规则，类型是 TileMode
  - TileMode 一共有 3 个值可选：CLAMP, MIRROR 和 REPEAT。CLAMP 会在端点之外延续端点处的颜色；MIRROR 是镜像模式；REPEAT 是重复模式


#### 构造方法参数
```
LinearGradient(float x0, float y0, float x1, float y1, int color0, int color1, Shader.TileMode tile)
```

#### demo

```
Shader shader = new LinearGradient(100, 100, 500, 500, Color.parseColor("#E91E63"),  
        Color.parseColor("#2196F3"), Shader.TileMode.CLAMP);
paint.setShader(shader);

...

canvas.drawCircle(300, 300, 200, paint);
```

---

#### 2.1.2.2 RadialGradient：辐射渐变
- 从中心向周围辐射状的渐变

#### 构造方法参数
- centerX、centerY：辐射中心的坐标
- radius：辐射半径 
- centerColor：辐射中心的颜色 
- edgeColor：辐射边缘的颜色 
- tileMode：辐射范围之外的着色模式

```
RadialGradient(float centerX, float centerY, float radius, int centerColor, int edgeColor, TileMode tileMode)
```

#### demo

```
Shader shader = new RadialGradient(300, 300, 200, Color.parseColor("#E91E63"),  
        Color.parseColor("#2196F3"), Shader.TileMode.CLAMP);
paint.setShader(shader);

...

canvas.drawCircle(300, 300, 200, paint);  
```

---

#### 2.1.2.3 SweepGradient：扫描渐变
- 扫描渐变？

#### 构造方法参数： 
- cx、cy ：扫描的中心 
- color0：扫描的起始颜色 
- color1：扫描的终止颜色

```
SweepGradient(float cx, float cy, int color0, int color1)
```

#### demo
![image](paint1_page3.jpg)

```
Shader shader = new SweepGradient(300, 300, Color.parseColor("#E91E63"),  
        Color.parseColor("#2196F3"));
paint.setShader(shader);

...

canvas.drawCircle(300, 300, 200, paint); 
```

---

#### 2.1.2.4  BitmapShader
- 用 Bitmap 来着色，其实也就是用 Bitmap 的像素来作为图形或文字的填充

#### 构造方法参数 
- bitmap：用来做模板的 Bitmap 对象 
- tileX：横向的 TileMode 
- tileY：纵向的 TileMode

```
BitmapShader(Bitmap bitmap, Shader.TileMode tileX, Shader.TileMode tileY)
```

#### demo
```
Bitmap bitmap = BitmapFactory.decodeResource(getResources(), R.drawable.batman);  
Shader shader = new BitmapShader(bitmap, Shader.TileMode.CLAMP, Shader.TileMode.CLAMP);  
paint.setShader(shader);

...

canvas.drawCircle(300, 300, 200, paint);
```

---

#### 2.1.2.5 ComposeShader：混合着色器
- 两个 Shader 一起使用
- 注意：硬件加速下是不支持两个相同类型的 Shader 的

#### 构造方法参数
- shaderA, shaderB：两个相继使用的 Shader 
- mode: 两个 Shader 的叠加模式，即 shaderA 和 shaderB 应该怎样共同绘制。它的类型是  PorterDuff.Mode 

```
ComposeShader(Shader shaderA, Shader shaderB, PorterDuff.Mode mode)
```

#### demo
```
// 第一个 Shader：头像的 Bitmap
Bitmap bitmap1 = BitmapFactory.decodeResource(getResources(), R.drawable.batman);  
Shader shader1 = new BitmapShader(bitmap1, Shader.TileMode.CLAMP, Shader.TileMode.CLAMP);

// 第二个 Shader：从上到下的线性渐变（由透明到黑色）
Bitmap bitmap2 = BitmapFactory.decodeResource(getResources(), R.drawable.batman_logo);  
Shader shader2 = new BitmapShader(bitmap2, Shader.TileMode.CLAMP, Shader.TileMode.CLAMP);

// ComposeShader：结合两个 Shader
Shader shader = new ComposeShader(shader1, shader2, PorterDuff.Mode.SRC_OVER);  
paint.setShader(shader);

...

canvas.drawCircle(300, 300, 300, paint);
```

---

#### 2.1.2.6 PorterDuff.Mode
- 用来指定两个图像共同绘制时的颜色策略的, 它是一个 enum
- PorterDuff.Mode 一共有 17 个
![image](paint1_page4.jpg)


- Alpha 合成 (Alpha Compositing)
![image](paint1_page5.jpg)


- 混合 (Blending)
![image](paint1_page6.jpg)


- 结论
  - Alpha 合成类的效果都比较直观，基本上可以使用简单的口头表达来描述它们的算法
  - 混合类的效果就相对抽象一些，只从效果图不太能看得出它们的着色算法，更看不出来它们有什么用
  - 对于 Alpha 合成类的操作，掌握他们，并在实际开发中灵活运用；而对于混合类的，你只要把它们的名字记住就好了，这样当某一天设计师告诉你「我要做这种混合效果」的时候，你可以马上知道自己能不能做，怎么做

---

### 2.2 setColorFilter(ColorFilter colorFilter)
- 在 Paint 里设置 ColorFilter ，使用的是 Paint.setColorFilter(ColorFilter filter) 方法。ColorFilter 并不直接使用，而是使用它的子类。它共有三个子类：LightingColorFilter、PorterDuffColorFilter 和 ColorMatrixColorFilter

#### 2.2.1 LightingColorFilter
- 模拟简单的光照效果的
- 构造方法是 LightingColorFilter(int mul, int add)
- 参数里的 mul 和  add 都是和颜色值格式相同的 int 值，其中 mul 用来和目标像素相乘，add 用来和目标像素相加

```
R' = R * mul.R / 0xff + add.R  
G' = G * mul.G / 0xff + add.G  
B' = B * mul.B / 0xff + add.B
```

- 一个「保持原样」的「基本 LightingColorFilter 」，mul 为 0xffffff，add 为 0x000000(也就是0)
```
R' = R * 0xff / 0xff + 0x0 = R // R' = R  
G' = G * 0xff / 0xff + 0x0 = G // G' = G  
B' = B * 0xff / 0xff + 0x0 = B // B' = B
```

- 可以把红色屏蔽掉
```
R' = R * 0x0 / 0xff + 0x0 = 0 // 红色被移除  
G' = G * 0xff / 0xff + 0x0 = G  
B' = B * 0xff / 0xff + 0x0 = B
```

---

#### 2.2.2 PorterDuffColorFilter
- PorterDuffColorFilter 的作用是使用一个指定的颜色和一种指定的 PorterDuff.Mode 来与绘制对象进行合成
- 构造方法是 PorterDuffColorFilter(int color, PorterDuff.Mode mode)，其中的 color 参数是指定的颜色， mode 参数是指定的 PorterDuff.Mode
- 和 ComposeShader 不同的是，PorterDuffColorFilter 作为一个 ColorFilter，只能指定一种颜色作为源，而不是一个 Bitmap

---

#### 2.2.3 ColorMatrixColorFilter
- ColorMatrixColorFilter 使用一个 ColorMatrix 来对颜色进行处理。 ColorMatrix 这个类，内部是一个 4x5 的矩阵。
```
[ a, b, c, d, e,
  f, g, h, i, j,
  k, l, m, n, o,
  p, q, r, s, t ]
```

- 通过计算， ColorMatrix 可以把要绘制的像素进行转换
```
R’ = a*R + b*G + c*B + d*A + e;  
G’ = f*R + g*G + h*B + i*A + j;  
B’ = k*R + l*G + m*B + n*A + o;  
A’ = p*R + q*G + r*B + s*A + t;
```

- ColorMatrix 有一些自带的方法可以做简单的转换，例如可以使用 setSaturation(float sat) 来设置饱和度
- [参考实例](https://github.com/chengdazhi/StyleImageView)

---

### 2.3 setXfermode(Xfermode xfermode)
- 它处理的是「当颜色遇上 View」的问题
- 以绘制的内容作为源图像，以 View 中已有的内容作为目标图像，选取一个  PorterDuff.Mode 作为绘制内容的颜色处理方案。
- 创建 Xfermode 的时候其实是创建的它的子类  PorterDuffXfermode。Xfermode 只有一个子类。

```
Xfermode xfermode = new PorterDuffXfermode(PorterDuff.Mode.DST_IN);

...

canvas.drawBitmap(rectBitmap, 0, 0, paint);   // 画方  
paint.setXfermode(xfermode);   // 设置 Xfermode  
canvas.drawBitmap(circleBitmap, 0, 0, paint); // 画圆  
paint.setXfermode(null);       // 用完及时清除 Xfermode
```

---

#### 2.3.1 Xfermode 注意事项
- 使用离屏缓冲（Off-screen Buffer）

```
// saveLayer() 可以做短时的离屏缓冲
int saved = canvas.saveLayer(null, null, Canvas.ALL_SAVE_FLAG);

canvas.drawBitmap(rectBitmap, 0, 0, paint);   // 画方
paint.setXfermode(xfermode);     // 设置 Xfermode
canvas.drawBitmap(circleBitmap, 0, 0, paint); // 画圆
paint.setXfermode(null);         // 用完及时清除 Xfermode

canvas.restoreToCount(saved);
```

- View.setLayerType()
  - View.setLayerType() 是直接把整个 View 都绘制在离屏缓冲中。setLayerType(LAYER_TYPE_HARDWARE) 是使用 GPU 来缓冲，setLayerType(LAYER_TYPE_SOFTWARE) 是直接直接用一个 Bitmap 来缓冲。

- 控制好透明区域
  - 注意控制它的透明区域不要太小，要让它足够覆盖到要和它结合绘制的内容，否则得到的结果很可能不是你想要的

![image](paint1_page7.jpg)

---

## 3. 效果
### 3.1 抗锯齿、填充/轮廓、线条宽度等
- setAntiAlias (boolean aa) 设置抗锯齿，默认是关闭的
- setStyle(Paint.Style style)
  - 补充：针对 Style.STROKE，需要注意 Android 会将 STROKE 的宽度均分两半，一半绘制在圆外，一半绘制在圆内。所以绘制的时候需要注意坐标位置，不然会导致描边被截断，出现绘制不完整。
  ![image](paint1_page22.png)
- 线条形状
  - setStrokeWidth(float width)
    - 默认情况下，线条宽度为 0，但你会发现，这个时候它依然能够画出线，线条的宽度为 1 像素。那么它和线条宽度为 1 有什么区别呢？其实这个和后面要讲的一个「几何变换」有关：你可以为 Canvas 设置 Matrix 来实现几何变换（如放大、缩小、平移、旋转），在几何变换之后 Canvas 绘制的内容就会发生相应变化，包括线条也会加粗，例如 2 像素宽度的线条在 Canvas 放大 2 倍后会被以 4 像素宽度来绘制。而当线条宽度被设置为 0 时，它的宽度就被固定为 1 像素，就算 Canvas 通过几何变换被放大，它也依然会被以 1 像素宽度来绘制。Google 在文档中把线条宽度为 0 时称作「hairline mode（发际线模式）」。
  - setStrokeCap(Paint.Cap cap)
    - 设置线头的形状。线头形状有三种：BUTT 平头、ROUND 圆头、SQUARE 方头。默认为 BUTT
    - 当线条的宽度是 1 像素时，这三种线头的表现是完全一致的，全是 1 个像素的点；而当线条变粗的时候，它们就会表现出不同的样子
![image](paint1_page8.jpg)

- setStrokeJoin(Paint.Join join)
  - 设置拐角的形状。有三个值可以选择：MITER 尖角、 BEVEL 平角和 ROUND 圆角。默认为 MITER
![image](paint1_page9.jpg)


- setStrokeMiter(float miter)
  - 这个方法是对于 setStrokeJoin() 的一个补充，它用于设置 MITER 型拐角的延长线的最大值。
![image](paint1_page10.jpg)


- 为了避免意料之外的过长的尖角出现， MITER 型连接点有一个额外的规则：当尖角过长时，自动改用  BEVEL 的方式来渲染连接点
- 至于多尖的角属于过于尖，尖到需要转为使用 BEVEL 来绘制，则是由一个属性控制的，而这个属性就是  setStrokeMiter(miter) 方法中的 miter 参数。miter 参数是对于转角长度的限制，具体来讲，是指尖角的外缘端点和内部拐角的距离与线条宽度的比。
![image](paint1_page11.jpg)

---

### 3.2 色彩优化
#### 两个方法
- setDither(boolean dither) 和 setFilterBitmap(boolean filter) 。它们的作用都是让画面颜色变得更加「顺眼」，但原理和使用场景是不同的

#### setDither(boolean dither)
- 设置图像的抖动
  - 所谓抖动（注意，它就叫抖动，不是防抖动，也不是去抖动，有些人在翻译的时候自作主张地加了一个「防」字或者「去」字，这是不对的），是指把图像从较高色彩深度（即可用的颜色数）向较低色彩深度的区域绘制时，在图像中有意地插入噪点，通过有规律地扰乱图像来让图像对于肉眼更加真实的做法。
  - 在实际的应用场景中，抖动更多的作用是在图像降低色彩深度绘制时，避免出现大片的色带与色块。
  - paint.setDither(true); // 一行代码

#### setFilterBitmap(boolean filter)
- 设置是否使用双线性过滤来绘制 Bitmap
  - 图像在放大绘制的时候，默认使用的是最近邻插值过滤，这种算法简单，但会出现马赛克现象；而如果开启了双线性过滤，就可以让结果图像显得更加平滑。
  - paint.setFilterBitmap(true); // 一行代码

![image](paint1_page12.jpg)

---

### 3.3 setPathEffect(PathEffect effect)
- 使用 PathEffect 来给图形的轮廓设置效果
- 6 种 PathEffect

#### 3.3.1 PathEffect
- 分为两类，单一效果的 CornerPathEffect、DiscretePathEffect、DashPathEffect、PathDashPathEffect 和组合效果的 SumPathEffect、ComposePathEffect

#### a. CornerPathEffect
- 把所有拐角变成圆角
  - 构造方法 CornerPathEffect(float radius) 的参数 radius 是圆角的半径
```
PathEffect pathEffect = new CornerPathEffect(20);  
paint.setPathEffect(pathEffect);
...
canvas.drawPath(path, paint);
```

![image](paint1_page13.jpg)

#### b. DiscretePathEffect
- 把线条进行随机的偏离，让轮廓变得乱七八糟
  - DiscretePathEffect 具体的做法是，把绘制改为使用定长的线段来拼接，并且在拼接的时候对路径进行随机偏离。它的构造方法 DiscretePathEffect(float segmentLength, float deviation) 的两个参数中， segmentLength 是用来拼接的每个线段的长度， deviation 是偏离量。

```
PathEffect pathEffect = new DiscretePathEffect(20, 5);  
paint.setPathEffect(pathEffect);
...
canvas.drawPath(path, paint);
```

![image](paint1_page14.jpg)

#### c. DashPathEffect
- 使用虚线来绘制线条
  - 构造方法 DashPathEffect(float[] intervals, float phase) 中， 第一个参数 intervals 是一个数组，它指定了虚线的格式：数组中元素必须为偶数（最少是 2 个），按照「画线长度、空白长度、画线长度、空白长度」……的顺序排列，例如示例代码中的 20, 5, 10, 5 就表示虚线是按照「画 20 像素、空 5 像素、画 10 像素、空 5 像素」的模式来绘制；第二个参数 phase 是虚线的偏移量。

```
PathEffect pathEffect = new DashPathEffect(new float[]{20, 10, 5, 10}, 0);  
paint.setPathEffect(pathEffect);
...
canvas.drawPath(path, paint);
```

![image](paint1_page15.jpg)

#### d. PathDashPathEffect
- 使用一个 Path 来绘制「虚线」
  - 它的构造方法 PathDashPathEffect(Path shape, float advance, float phase, PathDashPathEffect.Style style) 中， shape 参数是用来绘制的 Path；advance 是两个相邻的 shape 段之间的间隔，不过注意，这个间隔是两个 shape 段的起点的间隔，而不是前一个的终点和后一个的起点的距离；phase 和 DashPathEffect 中一样，是虚线的偏移；最后一个参数 style，是用来指定拐弯改变的时候 shape 的转换方式。
  - style 的类型为 PathDashPathEffect.Style，是一个 enum ，具体有三个值：TRANSLATE：位移、ROTATE：旋转、MORPH：变体

```
Path dashPath = ...; // 使用一个三角形来做 dash  
PathEffect pathEffect = new PathDashPathEffect(dashPath, 40, 0, PathDashPathEffectStyle.TRANSLATE);
paint.setPathEffect(pathEffect);
...
canvas.drawPath(path, paint);
```

![image](paint1_page16.jpg)

#### e. SumPathEffect
- 一个组合效果类的 PathEffect。它的行为特别简单，就是分别按照两种 PathEffect 分别对目标进行绘制

```
PathEffect dashEffect = new DashPathEffect(new float[]{20, 10}, 0);  
PathEffect discreteEffect = new DiscretePathEffect(20, 5); 
pathEffect = new SumPathEffect(dashEffect, discreteEffect);
...
canvas.drawPath(path, paint);
```

![image](paint1_page17.jpg)

#### f. ComposePathEffect
- 也是一个组合效果类的 PathEffect。不过它是先对目标 Path 使用一个 PathEffect，然后再对这个改变后的 Path 使用另一个 PathEffect。
  - 构造方法 ComposePathEffect(PathEffect outerpe, PathEffect innerpe) 中的两个 PathEffect 参数，innerpe 是先应用的，outerpe 是后应用的。所以上面的代码就是「先偏离，再变虚线」。

```
PathEffect dashEffect = new DashPathEffect(new float[]{ 20, 10 }, 0);
PathEffect discreteEffect = new DiscretePathEffect(20, 5);
pathEffect = new ComposePathEffect(dashEffect, discreteEffect);
...
canvas.drawPath(path, paint);
```

![image](paint1_page18.jpg)

---

### 3.3.2 注意事项：PathEffect
- 在有些情况下不支持硬件加速，需要关闭硬件加速才能正常使用
  - Canvas.drawLine() 和 Canvas.drawLines() 方法画直线时，setPathEffect() 是不支持硬件加速的
  - PathDashPathEffect 对硬件加速的支持也有问题，所以当使用 PathDashPathEffect 的时候，最好也把硬件加速关了
  
---

### 3.4  setShadowLayer(float radius, float dx, float dy, int shadowColor)
- 在之后的绘制内容下面加一层阴影
- 方法的参数里，radius 是阴影的模糊范围；dx、dy 是阴影的偏移量；shadowColor 是阴影的颜色
- 如果要清除阴影层，使用 clearShadowLayer()
- 注意：
  - 在硬件加速开启的情况下，setShadowLayer() 只支持文字的绘制，文字之外的绘制必须关闭硬件加速才能正常绘制阴影
  - 如果 shadowColor 是半透明的，阴影的透明度就使用 shadowColor 自己的透明度；而如果  shadowColor 是不透明的，阴影的透明度就使用 paint 的透明度
```
paint.setShadowLayer(10, 0, 0, Color.RED);
...
canvas.drawText(text, 80, 300, paint);
```

---

### 3.5 setMaskFilter(MaskFilter maskfilter)
- 设置的是在绘制层上方的附加效果
- 基于整个画面来进行过滤
- MaskFilter 有两种： BlurMaskFilter 和 EmbossMaskFilter

#### BlurMaskFilter：模糊效果的 MaskFilter
- 构造方法 BlurMaskFilter(float radius, BlurMaskFilter.Blur style) 中， radius 参数是模糊的范围， style 是模糊的类型。
- style 一共有四种: NORMAL: 内外都模糊绘制、SOLID: 内部正常绘制，外部模糊、INNER: 内部模糊，外部不绘制、OUTER: 内部不绘制，外部模糊

```
Path dashPath = ...; // 使用一个三角形来做 dash  
PathEffect pathEffect = new PathDashPathEffect(dashPath, 40, 0, PathDashPathEffectStyle.TRANSLATE);
paint.setPathEffect(pathEffect);
...
canvas.drawPath(path, paint);
```

![image](paint1_page19.jpg)

#### EmbossMaskFilter：浮雕效果的 MaskFilter
- 构造方法  EmbossMaskFilter(float[] direction, float ambient, float specular, float blurRadius) 的参数里， direction 是一个 3 个元素的数组，指定了光源的方向； ambient 是环境光的强度，数值范围是 0 到 1； specular 是炫光的系数； blurRadius 是应用光线的范围

---

### 3.6 获取绘制的 Path
- 这组方法做的事是，根据 paint 的设置，计算出绘制 Path 或文字时的实际 Path。
- 什么叫「实际 Path」？ Path 就是 Path，这加上个「实际」是什么意思？
- 文字的 Path ？文字还有 Path？

#### getFillPath(Path src, Path dst)
- 所谓实际 Path，指的就是 drawPath() 的绘制内容的轮廓，要算上线条宽度和设置的 PathEffect。
- 默认情况下（线条宽度为 0、没有 PathEffect），原 Path 和实际 Path 是一样的；而在线条宽度不为 0 （并且模式为 STROKE 模式或 FLL_AND_STROKE ），或者设置了 PathEffect 的时候，实际 Path 就和原 Path 不一样了
- 通过 getFillPath(src, dst) 方法就能获取这个实际 Path。方法的参数里，src 是原 Path ，而 dst 就是实际 Path 的保存位置。 getFillPath(src, dst) 会计算出实际 Path，然后把结果保存在 dst 里。

![image](paint1_page20.jpg)

#### getTextPath(String text, int start, int end, float x, float y, Path path) / getTextPath(char[] text, int index, int count, float x, float y, Path path)
- 文字的绘制，虽然是使用 Canvas.drawText()  方法，但其实在下层，文字信息全是被转化成图形，对图形进行绘制的。getTextPath() 方法，获取的就是目标文字所对应的 Path 。这个就是所谓「文字的 Path」。

![image](paint1_page21.jpg)

---

## 4. drawText
- [见 Paint 学习笔记（下）](https://github.com/hatake8607/learning/blob/master/Android/Android%20%E5%9F%BA%E7%A1%80/Paint%20%E5%AD%A6%E4%B9%A0%E7%AC%94%E8%AE%B0/Paint%20%E5%AD%A6%E4%B9%A0%E7%AC%94%E8%AE%B0%EF%BC%88%E4%B8%8B%EF%BC%89.ipynb)

---

## 5. 初始化类
### reset()
- 重置 Paint 的所有属性为默认值。相当于重新 new 一个，不过性能当然高一些

### set(Paint src)
- 把 src 的所有属性全部复制过来。相当于调用 src 所有的 get 方法，然后调用这个 Paint 的对应的  set 方法来设置它们

### setFlags(int flags)
- 批量设置 flags。相当于依次调用它们的 set 方法
- setFlags(flags) 对应的 get 方法是 int getFlags()

```
paint.setFlags(Paint.ANTI_ALIAS_FLAG | Paint.DITHER_FLAG);
```

---